In [18]:
import json, os, shutil, tqdm, skimage.draw

datasets = [{
#     'json_file': "/Users/skabbit/Downloads/Katya+Sasha.json",
#     'image_folder': "/Users/skabbit/Projects/backlash/backlash/datasets/usa-police"
# }, {
    'json_file': "/Users/skabbit/Downloads/backlash-auto-1 (3).json",
    'image_folder': "/Users/skabbit/Downloads/backlash-video-good/"
}]

result_folder = "/Users/skabbit/Downloads/backlash-video-manual-2classes/"
shutil.rmtree(result_folder, ignore_errors=True)
os.makedirs(result_folder, exist_ok=True)
os.makedirs(os.path.join(result_folder, "train"), exist_ok=True)
os.makedirs(os.path.join(result_folder, "val"), exist_ok=True)


In [ ]:
image_list = []

for dataset in datasets:
    json_file = dataset['json_file']
    image_folder = dataset['image_folder'] # or os.path.dirname(json_file)

    data = json.load(open(json_file))
    if "_via_img_metadata" in data:
        # switch to project JSON format
        data = data['_via_img_metadata'] 
    print("Loaded images:", len(data))

    for key in tqdm.tqdm(list(data.keys())):
        regions = data[key]['regions']
        
        # # filter by category_id==1
        # regions = list(filter(lambda x: 'category_id' not in x['region_attributes'] or x['region_attributes']['category_id'] == '1', regions))
        
        # filter out small regions (mask has less then 1000 pixels)
        regions = list(filter(lambda x: len(skimage.draw.polygon(x['shape_attributes']['all_points_y'], x['shape_attributes']['all_points_x'])[0]) > 1000, regions))
        
        if regions:
            data[key]['regions'] = regions
            data[key]['path'] = image_folder
            image_list.append({key: data[key]})
        
    print("Filtered images:", len(image_list))


Loaded images: 440


 25%|██▌       | 112/440 [01:10<03:07,  1.75it/s]

In [ ]:
# randomly split image_folder in proportion 7/1
import random
random.shuffle(image_list)

out = {
    "train": dict(),
    "val": dict(),
}

os.makedirs(os.path.join(image_folder, "train"), exist_ok=True)
os.makedirs(os.path.join(image_folder, "val"), exist_ok=True)

val_size = int(len(image_list) / 8)

for obj in tqdm.tqdm(image_list):
    dest = "val" if len(out['val']) < val_size else "train"
    out[dest].update(obj)
    filename = list(obj.values())[0]['filename']
    path = list(obj.values())[0]['path']
    result_filename = os.path.join(result_folder, dest, filename)
    if os.path.exists(result_filename):
        fn, ext = os.path.splitext(result_filename)
        size = os.path.getsize(result_filename)
        result_filename = "%s-%i%s" % (fn, size, ext)

    shutil.copyfile(
        os.path.join(image_folder, filename), 
        result_filename)

# create folders train/val

print("train size:", len(out['train']))
print("val size:", len(out['val']))

In [ ]:
for dest in out.keys():
    json.dump(
        dict(sorted(out[dest].items())), 
        open(os.path.join(result_folder, dest, "via_region_data.json"), "w"))
